In [1]:
%load_ext autoreload
%autoreload 2
%pdb
import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG)
logging.getLogger().setLevel(logging.INFO)

Automatic pdb calling has been turned ON


In [2]:
import sys
sys.path.insert(0, '/home/ubuntu/Documents/infembed')
#sys.path.insert(0, '/home/ubuntu/Documents/infembed/infembed')
# sys.path.insert(0, '/home/ubuntu/Documents/infembed/infembed')
# sys.path.insert(0, '/home/ubuntu/Documents/infembed/data')
from data._core.spotcheck import get_spotcheck_dataloader, get_blindspots_df
# sys.path.insert(0, '/home/ubuntu/Documents/infembed/models')
#sys.path.insert(0, '/home/ubuntu/Documents/infembed/')
# sys.path.insert(0, )
from infembed.embedder._core.fast_kfac_embedder import FastKFACEmbedder
import torchvision
from torch.utils.data import Subset, DataLoader, default_collate, Dataset
from torchvision.models import ResNet18_Weights, resnet18
import torch.nn as nn
from infembed.clusterer._core.sklearn_clusterer import SklearnClusterer
from infembed.clusterer._core.faiss_clusterer import FAISSClusterer
from infembed.clusterer._core.rule_clusterer import RuleClusterer
from sklearn.cluster import KMeans
from tqdm import tqdm
import pandas as pd
import torch
torch.multiprocessing.set_start_method('spawn')
from typing import List
from infembed.utils.common import Data
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Callable
from torch import Tensor
from typing import Tuple
from models._utils.common import init_linear
from lightning.pytorch.callbacks import EarlyStopping
import lightning.pytorch as pl
import lightning as L
import torch.nn.functional as F
import numpy as np
import pandas as pd
from data._utils.common import LimitIterableDataset
from data._core.tinystories import TinyStoriesCollateFn, TinyStoriesDataset
from data._utils.common import IterableDatasetToDataset

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.


### figure out device to compute embeddings on ###

In [3]:
DEVICE = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print('device:', DEVICE)

device: cuda:0


### define data
We will define the following:
- `eval_dataset`: `Dataset` for evaluation data.  This is used to retrieve individual examples for displaying.  This should be the same data for which the embeddings we load later were computed
- `eval_dataloader`: `DataLoader` for evaluation data.  This is used to compute metadata for the evaluation data

In [5]:
eval_dataset = IterableDatasetToDataset(
    LimitIterableDataset(
        dataset=TinyStoriesDataset(
            "/home/ubuntu/Documents/infembed/files/tinystories/TinyStoriesV2-GPT4-valid.txt"
        ),
    ),
)

eval_dataloader = DataLoader(
    dataset=eval_dataset,
    collate_fn=TinyStoriesCollateFn(duplicate=True, device=DEVICE),
    batch_size=4,
)

### define model ###
only used to get metadata

In [7]:
from models._utils.common import HuggingfaceWrapperModel, load_model
from transformers import AutoModelForCausalLM

model = load_model(
    model=HuggingfaceWrapperModel(
        model=AutoModelForCausalLM.from_pretrained(
            pretrained_model_name_or_path='roneneldan/TinyStories-33M',
        )
    ),
    device=DEVICE,
    eval=True,
)
model

HuggingfaceWrapperModel(
  (model): GPTNeoForCausalLM(
    (transformer): GPTNeoModel(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(2048, 768)
      (drop): Dropout(p=0.0, inplace=False)
      (h): ModuleList(
        (0-3): 4 x GPTNeoBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPTNeoAttention(
            (attention): GPTNeoSelfAttention(
              (attn_dropout): Dropout(p=0.0, inplace=False)
              (resid_dropout): Dropout(p=0.0, inplace=False)
              (k_proj): Linear(in_features=768, out_features=768, bias=False)
              (v_proj): Linear(in_features=768, out_features=768, bias=False)
              (q_proj): Linear(in_features=768, out_features=768, bias=False)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPTNeoMLP(
            (c_fc): Linear(in_fe

### load embeddings for evaluation data ###

In [8]:
embeddings_path = '/home/ubuntu/Documents/infembed/examples/tinystories/hydra_outputs/run_embedder/last_layer/dim=500/blocks=5/embeddings.pt'
embeddings = torch.load(open(embeddings_path, 'rb'))
embeddings.shape

torch.Size([27630, 479])

optionally, we can only keep some elements of the embedding (relevant for `ArnoldiEmbedder`)

In [9]:
DIMS_TO_KEEP = None
if DIMS_TO_KEEP is not None:
    # embeddings = embeddings[:, -DIMS_TO_KEEP:]
    embeddings = embeddings[:, :DIMS_TO_KEEP]

optionally, we can normalize the embeddings to have the same norm

In [10]:
NORMALIZE = True
if NORMALIZE:
    embeddings = embeddings / torch.linalg.norm(embeddings, dim=1)[:, None]

In [11]:
if True:
    # already assume embeddings fit in memory, so no point in using incremental methods
    pass

we then package them into a `Data` instance, which contains all kinds of data that could possibly be used to do the subsequent clustering, i.e. including tabular metadata as well.

In [11]:
data = Data(embeddings=embeddings)

### define clusterer ###

In [17]:
if False:
    clusterer = SklearnClusterer(sklearn_clusterer=KMeans(n_clusters=25))
if True:
    clusterer = FAISSClusterer(k=25, spherical=True)

### do the clustering ###

In [18]:
clusters = clusterer.fit_predict(data)

### define ways to display clusters ###
these will all be functions whose input is a list of list of indices in the evaluation dataset

In [20]:
from infembed.visualization._core.common import (
    DisplayMetadata,
    DisplayPIL,
    DisplayPredictionAndLabels,
    DisplaySingleExamples,
    PerClusterDisplayer,
    DisplayAccuracy,
    SingleExampleDisplayer,
)

class PrintDisplay(SingleExampleDisplayer):
    def __call__(self, i, data):
        print(data.dataset[i])


displayers = [
    PerClusterDisplayer(
        [
            # DisplayAccuracy(prediction_col="prediction_label", label_col="label"),
            # DisplayPredictionAndLabels(
            #     prediction_col="prediction_label_name", label_col="label_name"
            # ),
            DisplaySingleExamples(
                [
                    # DisplayMetadata(["label_name", "prediction_label_name"]),
                    PrintDisplay(),
                ],
                limit=3,
            ),
        ]
    )
]

### display the clusters ###

In [21]:
for displayer in displayers:
    displayer(clusters, data)

cluster #0
Once upon a time, a little bird lived in a small tree. The little bird loved to play with his friend, the squirrel. They liked to play with twigs. One day, the little bird was upset. He did not know where his friend was.
The bird looked all around for his friend. He saw a big twig on the ground. The bird thought, "Maybe my friend is hiding behind the twig." The bird flew down to look. But his friend was not there. The bird was more upset now.
Just then, the squirrel jumped out from a bush. He had a big smile on his face. He said, "I was hiding to surprise you!" The bird was happy to see his friend. They played with the big twig together. The little bird learned that sometimes, when things seem bad, a happy surprise might be just around the corner.
Once upon a time, there was a little dog named Spot. Spot was a dirty dog because he liked to play in the mud. One day, Spot found a big tip on the ground. It was a very special day, and all his friends were going to celebrate.
Spo